# Headless Playground

*This mode is suitable to display the gui on remote machines (i.e. cannot set up the polyscope gui).*
Here we show how to use `Engine3DGRUT` with [NVIDIA kaolin's[(https://github.com/NVIDIAGameWorks/kaolin) viewer. 

Note: Other viewers can similarly interact with the engine.

## Engine Params

Point at your playground params here.

* *gs_object*: Path of pretrained 3dgrt checkpoint, as .pt / .ingp / .ply file.
* *mesh_assets*: Path to folder containing mesh assets of .obj or .glb format.
* *default_config*: Name of default config to use for .ingp, .ply files, or .pt files not trained with 3dgrt.

In [ ]:
gs_object = "3dgrut/runs/<YOUR_MODEL_HERE>/ckpt_last.pt"
mesh_assets_folder = "./assets"
default_config = "apps/colmap_3dgrt.yaml"

## Install additional requirements

In [ ]:
!pip install matplotlib ipywidgets --quiet

#### Add 3dgrut root to search path

In [ ]:
import sys
from pathlib import Path

notebook_path = Path().resolve()
root_path = notebook_path.parent
sys.path.append(str(root_path))

## 3DGRUT Playground Engine

#### Setup Headless Engine

In [ ]:
import os
import copy
import numpy as np
import torch
import torchvision.transforms.functional as F
import kaolin
from matplotlib import pyplot as plt

from threedgrut.utils.logger import logger
from threedgrut.gui.ps_extension import initialize_cugl_interop
from threedgrut_playground.utils.video_out import VideoRecorder
from threedgrut_playground.engine import Engine3DGRUT, OptixPrimitiveTypes

In [ ]:
engine = Engine3DGRUT(
    gs_object=gs_object,
    mesh_assets_folder=mesh_assets_folder,
    default_config=default_config
)

#### Configure 3DGRUT Engine

In [ ]:
# Configure rendering settings
engine.camera_type = 'Pinhole'
engine.camera_fov = 60.0
engine.use_spp = True
engine.antialiasing_mode = '8x MSAA'

# Remove initial glass sphere from scene
for mesh_name in list(engine.primitives.objects.keys()):
    engine.primitives.remove_primitive(mesh_name)

# Add a glass 'Armadillo' to the scene
# Get the asset with download_assets.sh script
engine.primitives.add_primitive(
    geometry_type='Armadillo',
    primitive_type=OptixPrimitiveTypes.GLASS,
    device='cuda'
)

## Render Single Image

The following block shows how to generate a single screenshot using the engine.

In [ ]:
# You can also create a camera from (eye, at, up) or 4x4 view-matrix
# See here: https://github.com/NVIDIAGameWorks/kaolin/blob/master/examples/recipes/camera/camera_init_simple.py

# Create a camera programatically and position it
camera = kaolin.render.easy_render.default_camera(512).cuda()
camera.move_forward(-1.5)
camera.move_up(1.5)
camera.rotate(yaw=1.0, pitch=0.5, roll=2.5)

In [ ]:
# Render a full quality frame
framebuffer = engine.render(camera)
rgba_buffer = torch.cat([framebuffer['rgb'], framebuffer['opacity']], dim=-1)

# Display
chw_buffer = rgba_buffer[0].permute(2, 0, 1)
img = F.to_pil_image(chw_buffer)
plt.imshow(img)
plt.axis('off')
plt.show()

## Interactive Renderer

The following block shows how to use kaolin's internal viewer to drive the 3DGRUT engine with user interaction.

#### Set up widgets

Set up some checkboxes, dropdowns and slider to control simple engine functionality.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

aa_checkbox = widgets.Checkbox(
    value=engine.use_spp,
    description='Toggle Antialiasing'
)

aa_mode_combo = widgets.Dropdown(
    options=engine.ANTIALIASING_MODES,
    value='4x MSAA',
    description='AA Mode'
)

denoiser_checkbox = widgets.Checkbox(
    value=engine.use_optix_denoiser,
    description='Toggle Optix Denoiser'
)

spp_slider = widgets.IntSlider(
    value=engine.spp.spp,
    min=1,
    max=64,
    step=1,
    orientation='horizontal',
    description='AA SPP',
    disabled=(engine.spp.mode == 'msaa')
)

def on_change(change):
    engine.use_spp = aa_checkbox.value
    
    engine.antialiasing_mode = aa_mode_combo.value
    if engine.antialiasing_mode == '4x MSAA':
        engine.spp.mode = 'msaa'
        engine.spp.spp = 4
    elif engine.antialiasing_mode == '8x MSAA':
        engine.spp.mode = 'msaa'
        engine.spp.spp = 8
    elif engine.antialiasing_mode == '16x MSAA':
        engine.spp.mode = 'msaa'
        engine.spp.spp = 16
    elif engine.antialiasing_mode == 'Quasi-Random (Sobol)':
        engine.spp.mode = 'low_discrepancy_seq'
        engine.spp.spp = spp_slider.value
    else:
        raise ValueError('unknown antialiasing mode')
        
    engine.spp.reset_accumulation()
    engine.use_optix_denoiser = denoiser_checkbox.value

    spp_slider.value = engine.spp.spp
    spp_slider.disabled = (engine.spp.mode == 'msaa')
    visualizer.render_update()

aa_checkbox.observe(on_change, names='value')
aa_mode_combo.observe(on_change, names='value')
denoiser_checkbox.observe(on_change, names='value')
spp_slider.observe(on_change, names='value')

#### Plug 3DGRUT Engine to Canvas

In [ ]:
def render(in_cam, **kwargs):
    # Called when the user stops interacting, to generate a high quality frame
    is_first_pass = engine.is_dirty(in_cam)
    framebuffer = engine.render_pass(in_cam, is_first_pass=True)
    while engine.has_progressive_effects_to_render():
        framebuffer = engine.render_pass(in_cam, is_first_pass=False)

    rgba_buffer = torch.cat([framebuffer['rgb'], framebuffer['opacity']], dim=-1)
    rgba_buffer = torch.clamp(rgba_buffer, 0.0, 1.0)

    return (rgba_buffer[0] * 255).to(torch.uint8)

def fast_render(in_cam, **kwargs):
    # Called during interactions, disables effects for quick rendering
    framebuffer = engine.render_pass(in_cam, is_first_pass=True)
    rgba_buffer = torch.cat([framebuffer['rgb'], framebuffer['opacity']], dim=-1)
    rgba_buffer = torch.clamp(rgba_buffer, 0.0, 1.0)
    return (rgba_buffer[0] * 255).to(torch.uint8)

# Create initial camera
camera = kaolin.render.easy_render.default_camera(512)
camera.change_coordinate_system(
    torch.tensor([[1, 0, 0],
                  [0, 0, 1],
                  [0, -1, 0]]
))
camera = camera.cuda()
# Initialize renderer
visualizer = kaolin.visualize.IpyTurntableVisualizer(
    height=camera.height,
    width=camera.width,
    camera=copy.deepcopy(camera),
    render=render,
    fast_render=fast_render,
    max_fps=8,
    world_up_axis=1
)

# Show the canvas and callback listener
vbox = widgets.VBox([denoiser_checkbox, aa_checkbox, aa_mode_combo, spp_slider])
hbox = widgets.HBox([visualizer.canvas, vbox])
display(hbox, visualizer.out)

# OR without a GUI, its as simple as:
# visualizer.show()